In [1]:
import igraph as ig
import leidenalg
import pandas as pd
import networkx as nx

# === Cargar nodos y aristas ===
nodes_df = pd.read_csv("../data/networks/lab_nodes.csv")
edges_df = pd.read_csv("../data/networks/lab_edges.csv")

# === Construir grafo no dirigido ===
G = nx.Graph()
for _, row in nodes_df.iterrows():
    G.add_node(row["Id"], label=row["Label"], lab=row["Lab"])
for _, row in edges_df.iterrows():
    G.add_edge(row["Source"], row["Target"], weight=row["Weight"])

print(G.number_of_nodes())
print(G.number_of_edges())

# Convertir desde NetworkX
G_ig = ig.Graph.TupleList(G.edges(data=True), directed=False, weights=True)

# Añadir nodos aislados (que están en G pero no en G_ig)
nodos_desconectados = set(G.nodes()) - set(G_ig.vs["name"])
for node in nodos_desconectados:
    G_ig.add_vertex(name=node)
    
# Ejecutar Leiden con diferentes resolutions
resolutions = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0]

for res in resolutions:
    rb_partition = leidenalg.find_partition(
        G_ig,
        leidenalg.RBConfigurationVertexPartition,
        resolution_parameter=res
    )
    print(f"[RBConfig] Resolution={res} → modularidad={rb_partition.modularity:.4f}, comunidades={len(rb_partition)}")

# Comparativa con modularidad clásica (no parametrizable)
mod_partition = leidenalg.find_partition(
    G_ig,
    leidenalg.ModularityVertexPartition
)
print(f"[Modularity] clásico → modularidad={mod_partition.modularity:.4f}, comunidades={len(mod_partition)}")

partition = {}
for community_id, nodes in enumerate(mod_partition):
    for node in nodes:
        partition[G_ig.vs[node]["name"]] = community_id

for node in G.nodes():
    if node not in partition:
        partition[node] = -1  # comunidad especial para desconectados



29
66
[RBConfig] Resolution=0.25 → modularidad=0.3361, comunidades=3
[RBConfig] Resolution=0.5 → modularidad=0.4541, comunidades=4
[RBConfig] Resolution=0.75 → modularidad=0.5009, comunidades=5
[RBConfig] Resolution=1.0 → modularidad=0.5009, comunidades=5
[RBConfig] Resolution=1.25 → modularidad=0.5009, comunidades=5
[RBConfig] Resolution=1.5 → modularidad=0.5009, comunidades=5
[RBConfig] Resolution=2.0 → modularidad=0.4602, comunidades=7
[RBConfig] Resolution=2.5 → modularidad=0.4343, comunidades=8
[RBConfig] Resolution=3.0 → modularidad=0.3848, comunidades=10
[RBConfig] Resolution=3.5 → modularidad=0.3430, comunidades=11
[RBConfig] Resolution=4.0 → modularidad=0.2963, comunidades=13
[RBConfig] Resolution=5.0 → modularidad=0.2265, comunidades=16
[Modularity] clásico → modularidad=0.5009, comunidades=5


In [2]:
# === Buscar la mejor partición (mayor modularidad)
best_partition = None
best_modularity = -1
best_res = None

# Guardar en CSV
df_partition = pd.DataFrame.from_dict(partition, orient='index', columns=['leiden_community'])
df_partition.index.name = 'author_name'
df_partition.reset_index(inplace=True)
df_partition.to_csv("Lovaina/leiden_best_partition.csv", index=False)

In [23]:
from pyvis.network import Network
import networkx as nx
from IPython.display import display, IFrame


# Crear red
net = Network(height='700px', width='100%', notebook=True, directed=False)
net.barnes_hut()

net.set_options('''
var options = {
  "nodes": {
    "font": {
      "size": 16,
      "face": "Tahoma"
    },
    "shape": "dot"
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "interaction": {
    "hover": true,
    "tooltipDelay": 200
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -8000,
      "springLength": 150,
      "springConstant": 0.04,
      "damping": 0.7
    },
    "minVelocity": 0.75
  }
}
''')


# Añadir nodos
for node in G.nodes():
    community = partition.get(node, -1)
    net.add_node(
        node,
        label=node,  # Se mostrará junto al nodo
        title=f"{node} (Comunidad {community})",  # Se mostrará en tooltip
        group=community,
        size=20
    )


# Añadir aristas
for u, v, data in G.edges(data=True):
    net.add_edge(u, v, value=data.get('weight', 1))

# Exportar y mostrar inline
file_path = "red_ips_comunidades.html"
net.show(file_path)

# Mostrar en el notebook directamente
display(IFrame(src=file_path, width="100%", height="700px"))


red_ips_comunidades.html
